In [ ]:
#yes thank you again K
from spellchecker import SpellChecker # pip install pyspellchecker
import telebot # pip install pyTelegramBotAPI
from telebot import types
import time
import random

In [ ]:
token='TOKEN-MASKED-FOR-PRIVACY'
bot=telebot.TeleBot(token)

In [ ]:
# need to consider shared names> user_id will be useful but distinguishing user's with shared user_ids hmmmmmmmmmm
# cannot rely on username as ppl can have no username ; will show up as None which can affect usage
# userid is obvs good but numbers
# mb link userid dic to username dic?
# still can be improved, but will leave it for now
game_start=False
letter=''
game_type=None
chat_type=None

id_points_dic={}
id_name_dic={}

word_hist=[]
players=[]
@bot.message_handler(commands=['start'])
def start_message(m):
    global game_start; game_start=False
    bot.send_message(m.chat.id,'Hello and Welcome to the Letter game Bot!')
    time.sleep(1)
    markup=types.InlineKeyboardMarkup()
    start_b=types.InlineKeyboardButton('Starting letter',callback_data='start_game')
    end_b=types.InlineKeyboardButton('Ending letter',callback_data='end_game')
    markup.add(start_b)
    markup.add(end_b)
    bot.send_message(m.chat.id,'''Please select the type of game you wish to play''',reply_markup=markup)

@bot.callback_query_handler(func=lambda call: call.data.endswith('_game'))
def check_userid(m):
    bot.edit_message_reply_markup(m.message.chat.id, m.message.message_id)
    global game_type
    additional='' if str(m.message.chat.type).lower()=='private' else 'and your friends '
    if m.data.startswith('start'):
        bot.send_message(m.message.chat.id,f'''The aim of the game is simple; when the game begins,
a random alphabet will be given for you {additional}to come up with as many words that *BEGIN* with it.
    \nYou will have 60 seconds to type as many valid words possible; no repeats are allowed, so keep an eye out!''',parse_mode='Markdown')
        game_type=0
    else:
        bot.send_message(m.message.chat.id,f'''The aim of the game is simple; when the game begins,
a random alphabet will be given for you {additional}to come up with as many words that *END* with it.
    \nYou will have 60 seconds to type as many valid words possible; no repeats are allowed, so keep an eye out!''',parse_mode='Markdown')
        game_type=1
    if str(m.message.chat.type).lower()=='private':
        global game_start; game_start=True
    else:
        global chat_type;chat_type=1
    global letter; letter=''
    global players; players.clear()
    global word_hist; word_hist.clear()
    if str(m.message.chat.type).lower()=='private':
        name=m.from_user.first_name
        id=m.from_user.id
        id_points_dic[id]=0
        id_name_dic[id]=name
        markup=types.InlineKeyboardMarkup()
        ready_b=types.InlineKeyboardButton('Ready',callback_data='ready')
        markup.add(ready_b)
        bot.send_message(m.message.chat.id,f'When you\'re all set, click the "Ready" button below.',reply_markup=markup)
    else:
        bot.send_message(m.message.chat.id,f'To join the game,\nsimply type "/join".')  

@bot.message_handler(commands=['join'])
def join_message(m):
    global players
    if game_start or chat_type==1:
        time.sleep(1)
        user=m.from_user
        if user.id not in list(id_points_dic.keys()):
            id_points_dic[user.id]=0
            markup=types.InlineKeyboardMarkup()
            ready_b=types.InlineKeyboardButton('Ready',callback_data='ready')
            markup.add(ready_b)
            players.append(user.first_name)
            id_name_dic[user.id]=user.first_name
            all_users='\n'.join(players)            
            global def_m
            if len(players)==1:
                def_m=bot.send_message(m.chat.id,f'Players in the game:\n\n{all_users}\n\nOnce everyone has joined, click the "Ready" button below.',
                             reply_markup=markup)
            else:
                bot.delete_message(m.chat.id,def_m.message_id)
                def_m=bot.send_message(m.chat.id,f'Players in the game:\n\n{all_users}\n\nOnce everyone has joined, click the "Ready" button below.',
                             reply_markup=markup)
    else:
        pass

@bot.callback_query_handler(func=lambda call: call.data=='ready')
def game_begins(m):
    bot.edit_message_reply_markup(m.message.chat.id, m.message.message_id)
    global letter
    global game_type
    letter=random.choice([chr(l) for l in range(65,91)])
    if game_type==0:
        countdown=['Your',"Word's",'*First*','Letter','issssssssss'][::-1]
    else:
        countdown=['Your',"Word's",'*Last*','Letter','issssssssss'][::-1]
    for n in range(5,0,-1):
        bot.send_message(m.message.chat.id,f'{countdown[n-1]}',parse_mode='Markdown')
        time.sleep(1)
    time.sleep(1)
    bot.send_message(m.message.chat.id,f'The letter : *{letter}*',parse_mode='Markdown')
    global game_start; game_start=True
    global start 
    start= time.time()

@bot.message_handler(func=lambda msg: msg.text is not None and len(letter)>0)
def fn(msg):
    global id_points_dic
    global id_name_dic
    spell=SpellChecker()
    word=msg.text.upper()
    global letter
    global word_hist
    global game_type
    if time.time()-start>60:
        letter=''
        bot.send_message(msg.chat.id,f'And that\'s time!')
        global game_start; game_start=False
        max_score=max(id_points_dic.values())
        winner_id=[k for k,v in id_points_dic.items() if v==max_score]
        time.sleep(1)
        if str(msg.chat.type).lower()=='private': 
            bot.send_message(msg.chat.id,f'Congratulations {id_name_dic[winner_id[0]]}! You got {max_score} valid words!')
        else:
            if len(winner_id)==1:
                bot.send_message(msg.chat.id,f'The winner is {id_name_dic[winner_id[0]]} with {max_score} valid words!')
            else:
                all_to_last_names=list(map(lambda x: id_name_dic[x],winner_id[:-1]))
                all_to_last=', '.join(all_to_last_names)
                bot.send_message(msg.chat.id,f'''Looks like we have ties between {all_to_last} and {id_name_dic[winner_id[-1]]}
who all scored {max_score} valid words!!''')
        id_points_dic.clear()
        id_name_dic.clear()
        time.sleep(2.5)
        markup=types.InlineKeyboardMarkup()
        restart_start_b=types.InlineKeyboardButton('Starting Letter',callback_data='start_game')
        restart_end_b=types.InlineKeyboardButton('Ending Letter',callback_data='end_game')
        end_b=types.InlineKeyboardButton('End',callback_data='end')
        markup.add(restart_start_b)
        markup.add(restart_end_b)
        markup.add(end_b)
        bot.send_message(msg.chat.id,f'Want to have another run?',reply_markup=markup)
    else:
        if game_type==0:
            if str(word).startswith(letter) and word not in word_hist:
                try:
                    spell.correction(word).upper()
                except:
                    pass 
                else:
                    if spell.correction(word).upper()==word:
                        author=msg.from_user.id
                        id_points_dic[author]+=1 
                        word_hist.append(word)
            else:
                pass
        else:
            if str(word).endswith(letter) and word not in word_hist:
                try:
                    spell.correction(word).upper()
                except:
                    pass 
                else:
                    if spell.correction(word).upper()==word:
                        author=msg.from_user.id
                        id_points_dic[author]+=1 
                        word_hist.append(word)
            else:
                pass

@bot.callback_query_handler(func=lambda call: call.data=='end')
def finish_game(m):
    bot.edit_message_reply_markup(m.message.chat.id, m.message.message_id)
    bot.send_message(m.message.chat.id,'Hope you enjoyed the games played!\n\nThank you!')

while True:
    try:
        bot.polling(non_stop=True, interval=0)
    except Exception as e:
        print(e)
        time.sleep(5)
        continue